In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import ctx
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from ProjectTwo import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacUser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = 31487


#db = AnimalShelter(username, password)
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# DEBUG
# print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),height=250)),
    html.Center(html.B(html.H4('Welcome to the Animal Search Dashboard'))),
    html.Center(html.H5('Created by Mark Powers')),
    html.Hr(),
    
    html.Div([
    dcc.RadioItems(
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain and Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
            {'label': 'Reset', 'value': 'RESET'}
        ],
        id='radioButtonFilters',
        inline=True)
    ]),
    
    html.Hr(),
    
    
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable='single',
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    
    html.Div(className='row',
        style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    
    html.Br(),
    html.Hr(),

])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
     Output('datatable-id', "data"),
     Input('radioButtonFilters', 'value')
)

def updateTable(value):
    # This performs the qeury and filtering for Water Rescue Dogs
    if value == 'WR':
        df = pd.DataFrame.from_records(shelter.read({
            '$and':[{'breed':{'$in':['Chesa Bay Retr Mix',
                                     'Newfoundland Mix',
                                     'Labrador Retriever Mix']}},
                    {'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}]
        }))
        
    # This performs the query and filtering for Mountain and Wilderness  
    elif value == 'MWR':            
        df = pd.DataFrame.from_records(shelter.read({
            '$and':[{'breed':{'$in':['German Shepherd',
                                    'Alaskan Malamute',
                                    'Old English Sheepdog',
                                    'Rottweiler',
                                    'Siberian Husky']}},
                    {'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}]
        }))
        
    # Query and filter for Disaster Rescue and Tracking 
    elif value == 'DIT':             
        df = pd.DataFrame.from_records(shelter.read({
            '$and':[{'breed':{'$in':['German Shepherd',
                                    'Golden Retriever Mix',
                                    'Doberman Pinsch',
                                    'Rottweiler',
                                    'Bloodhound']}},
                    {'age_upon_outcome_in_weeks':{'$gte':20,'$lte':300}}]
        }))
        
    # Clears the filter results from the table
    elif value == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))
            
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')        


#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    #[Input('datatable-id', "derived_virtual_data")]
    [Input('radioButtonFilters', 'value')]
)
        
def update_graphs(value):
    
    # Changes the pie graph to show the numbers of each breed of water rescue dog
    if value == 'WR':
        dff = pd.DataFrame.from_records(shelter.read({
            '$and':[{'breed':{'$in':['Chesa Bay Retr Mix',
                                     'Newfoundland Mix',
                                     'Labrador Retriever Mix']}},
                    {'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}]
        }))
        names = dff['breed'].value_counts().keys().tolist()
        values = dff['breed'].value_counts().tolist()
        title = "Water Rescue"
        
    
    # Changes pie graph to show numbers of each breed of mountain/wilderness rescue dog
    elif value =='MWR':
        dff = pd.DataFrame.from_records(shelter.read({
            '$and':[{'breed':{'$in':['German Shepherd',
                                    'Alaskan Malamute',
                                    'Old English Sheepdog',
                                    'Rottweiler',
                                    'Siberian Husky']}},
                    {'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}]
        }))
        names = dff['breed'].value_counts().keys().tolist()
        values = dff['breed'].value_counts().tolist()
        title = "Mountain/Wilderness Rescue"

    
    # Changes graph for breeds of disaster and individual tracking
    elif value == 'DIT':             
        dff = pd.DataFrame.from_records(shelter.read({
            '$and':[{'breed':{'$in':['German Shepherd',
                                    'Golden Retriever Mix',
                                    'Doberman Pinsch',
                                    'Rottweiler',
                                    'Bloodhound']}},
                    {'age_upon_outcome_in_weeks':{'$gte':20,'$lte':300}}]
        }))
        names = dff['breed'].value_counts().keys().tolist()
        values = dff['breed'].value_counts().tolist()
        title = "Disaster and Individual Tracking"
    
    
    # Changes graph back to basic chart of all species of animal at animal center
    else:
        dff = pd.DataFrame.from_records(shelter.read({}))
        names = dff['animal_type'].value_counts().keys().tolist()
        values = dff['animal_type'].value_counts().tolist()
        title = 'Animals by Species'
        
    return [
        dcc.Graph(            
            figure = px.pie(data_frame= dff,
                           values = values,
                           names = names,
                           title = title,
                           color_discrete_sequence = px.colors.sequential.Plasma,
                           width=500, height=500)
        )    
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
        
def update_map(viewData, index):  
    #if viewData is None:
        #return
    #elif index is None:
        #return
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '400px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)

Connection Successful
Dash app running on http://127.0.0.1:10007/
